In [1]:
print("Trial graph for all the agents")

Trial graph for all the agents


IMPORTING ALL THE REQUIRED PACKAGES

In [2]:
from typing import TypedDict, List
import matplotlib.pyplot as plt

In [3]:
import research_agent  , question_generator_agent , coding_agent , research_2

C:\Users\pkgno\AppData\Local\Temp\ipykernel_35524\2486430593.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  import research_agent  , question_generator_agent , coding_agent , research_2


CREATING THE GRAPH STATE ---> this is a shared data source in the entire graph

In [4]:
class State(TypedDict):
    topic:str # this is the main concept/algorithm/idea to research and frame questions on 
    research_json: dict # this will hold the json output from the research agent
    question: dict # this is formatted data of the question that is generated by the question generator agent
    solution: dict # this field is the answer to the problem , need to be filled optionally

Creating a simple researcher node

In [5]:
from research_2 import JsonOutput , ResearchAgent


def research_node(state : State):
    """ this is a node for the researcher agent , to perform web search and curate
    information on the requested topic"""

    
    print("********* Researcher node is running**********")
    topic = state['topic']

    # create the research object and call the functions
    research_agent = ResearchAgent()
    output = research_agent.search(topic)
    print(f"the final output got from the research agent  is -->\n\n {output}" )

    # the final step is to add the data or any changes to the graph state
    return {"research_json": output}

Creating the question creator node

In [6]:
def question_crafter_node(state : State):
    """  This node is to be used for crafting the question from the research json object
    Here the key insights from the research will be transformed into a question after analyzing it and taking into consideration all the key technical details.
    """

    print("*********** Question Crafter node is running ****************")

    #1. take the research json object from the graph state
    research_json = state['research_json'] # this is the knowledge packet for creating the question

    # serialize this data
    import json
    from question_generator import QuestionGenerator
    question_creator = QuestionGenerator()
    research_string_data = json.dumps(research_json)
    generated_question = question_creator.generate_question(research_string_data)
    print(f"\n The question generated by the agent is : \n\n {generated_question}")
    #update the graph state with this question
    return {"question": generated_question.model_dump()}

Creating the coder agent 

In [7]:
def question_solver(state:State):
    """ This node is to be used for solving the question that is generated by the question generator agent
    This will involve analyzing the question and generating a solution based on the constraints given and generating the most optimal solution.
    """

    print("*********** Question Solver node is running ****************")

    #1. take the research json object from the graph state
    question = state['question'] 
    # this data needs to be converted into the pydantic model 
    from question_generator_agent import CodeQuestion # this is the data model
    import coding_agent
    formatted_question = CodeQuestion(**question)
    #now give this to the llm to solve
    solution = coding_agent.solve_question(formatted_question)
    print(f"\n The solution generated by the agent is : \n\n {solution}")
    
    return {"solution" : solution }

CREATING THE GRAPH 

In [8]:
# using the stateGraph of lang graph to create a 3 node graph to connect all the agents
from langgraph.graph import START , END , StateGraph

#1. initialize an empty graph with the state object
graph = StateGraph(State) # State is the shared data between all the nodes

#2. add nodes to the graph
graph.add_node("Researcher" , research_node)
graph.add_node("Question Generator" , question_crafter_node)
graph.add_node("Question Solver" , question_solver)

#3. add the edges 
graph.add_edge(START , "Researcher")
graph.add_edge("Researcher" , "Question Generator")
graph.add_edge("Question Generator" , "Question Solver")
graph.add_edge("Question Solver" , END)

#graph.add_edge("Researcher" , END)

complete_dsa_agent = graph.compile()

In [9]:
# printing the graph 

print(complete_dsa_agent.get_graph().print_ascii())

    +-----------+      
    | __start__ |      
    +-----------+      
           *           
           *           
           *           
    +------------+     
    | Researcher |     
    +------------+     
           *           
           *           
           *           
+--------------------+ 
| Question Generator | 
+--------------------+ 
           *           
           *           
           *           
  +-----------------+  
  | Question Solver |  
  +-----------------+  
           *           
           *           
           *           
      +---------+      
      | __end__ |      
      +---------+      
None


TESTING THE GRAPH WORK FLOW

In [ ]:
# --- FINAL TEST BLOCK ---
if __name__ == "__main__":
    import json

    # Define the initial state to kick off the graph
    initial_state = {
        "topic": "key concepts on binary search algorithm and most common questions on it"
    }

    print("🚀 Invoking the LangGraph Application...")
    
    # The .invoke() method runs the graph from the entry point to the end.
    final_state = complete_dsa_agent.invoke(initial_state)

    print("\n--- ✅ Graph Execution Complete ---")
    
    # The final state contains all the accumulated data.
    # We can now access the generated question and its solution.
    
    print("\n\n--- Generated Question ---")
    print(json.dumps(final_state['question'], indent=2))
    
    print("\n\n--- Generated Solution ---")
    print(json.dumps(final_state['solution'], indent=2))

🚀 Invoking the LangGraph Application...
********* Researcher node is running**********
Output of gemini api is : 
['The article "Top 10 Most Asked Binary Search Interview Questions" by Kirti Arora provides a comprehensive guide to common binary search problems encountered in technical interviews, emphasizing its O(log n) time complexity. It covers ten distinct problems, offering Java code solutions for each.\n\nThe questions addressed include:\n*   **Basic Binary Search:** Implementing the core algorithm to find a target in a sorted array, demonstrating both iterative and recursive approaches.\n*   **First and Last Position of an Element:** Modifying binary search to locate the starting and ending indices of a given target in a sorted array, requiring two separate binary search-like functions.\n*   **Search Insert Position:** Finding the index of a target in a sorted array, or where it would be inserted to maintain order if not present.\n*   **Kth Smallest Element in a Sorted Matrix:**

In [60]:
print(final_state['research_json'])

{
  "title": "Comprehensive Guide to Binary Search",
  "description": "Binary search is an efficient algorithm for finding a specific target element within a sorted array or list. It works by repeatedly dividing the search interval in half, eliminating half of the remaining elements with each comparison until the target is found or the search interval becomes empty.",
  "algorithm_steps": [
    {},
    {},
    {},
    {},
    {}
  ],
  "coding_implementations": [
    {}
  ],
  "specialized_implementations": [
    {},
    {},
    {},
    {},
    {},
    {},
    {},
    {},
    {}
  ],
  "benefits": [
    {},
    {},
    {},
    {}
  ]
}


TESTING INDIVIDUAL NODES 

In [61]:
if __name__ == "__main__":
    import json

    # 1. Create a mock initial state
    # This simulates the state of the graph right at the beginning.
    mock_initial_state = {
        "topic": "binary search algorithm and its key technical considerations"
    }

    # 2. Call our node function directly
    # This is our unit test.
    research_result = research_node(mock_initial_state)

    # 3. Print the result to verify
    # We expect this to be a dictionary with the key "research_json".
    print("\n--- Node Test Result ---")
    print(json.dumps(research_result, indent=2))

    # You can also check the keys to be sure
    if "research_json" in research_result:
        print("\n✅ Test Passed: The node returned the correct key.")
    else:
        print("\n❌ Test Failed: The node did not return the 'research_json' key.")

********* Researcher node is running**********
Output of gemini api is : 
['A binary search algorithm is an efficient method for finding a target value within a *sorted* array or list by repeatedly dividing the search interval in half. This "divide-and-conquer" approach offers a significant advantage over linear search, particularly for large datasets, as it reduces the number of comparisons needed. Its core mechanism involves comparing the target to the middle element: if they match, the search concludes; otherwise, the search continues in either the left or right half based on whether the target is smaller or larger than the middle element.\n\nTechnically, binary search requires the input data to be sorted and can be implemented either recursively or iteratively, necessitating a base case for termination. A crucial aspect of its efficiency is its logarithmic time complexity, O(log n), which makes it exponentially faster than linear search\'s O(n) for large datasets. However, implemen